In [1]:
%pip install yfinance
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as data

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/1b/0f/77716aa9dd84bb1aa5e93c87122af1de89697b6231f6d01d58d4e7c03c14/yfinance-0.2.36-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/314.6 kB ? eta -:--:--
     -- ---------------------------------- 20.5/314.6 kB 330.3 kB/s eta 0:00:01
     ----------------- -------------------- 143.4/314.6 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 314.6/314.6 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     --

ModuleNotFoundError: No module named 'yfinance'

In [2]:
start = '2010-01-01'
end = '2020-12-31'

df = yf.download('AAPL', start, end)

NameError: name 'yf' is not defined

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df = df.reset_index()
df.head()

In [ ]:
df = df.drop(['Date', 'Adj Close'], axis = 1)
df.head()

In [ ]:
plt.plot(df.Close)

In [ ]:
df

In [ ]:
ma100 = df.Close.rolling(100).mean()
ma100

In [ ]:
plt.figure(figsize = (12,6))
plt.plot(df.Close)
plt.plot(ma100, 'r')


In [ ]:
ma200 = df.Close.rolling(200).mean()
ma200

In [ ]:
plt.figure(figsize = (12,6))
plt.plot(df.Close)
plt.plot(ma100, 'r')
plt.plot(ma200, 'g')

In [ ]:
df.shape

In [ ]:
# Splitting data into Training and Testing

data_training = pd.DataFrame(df['Close'][0:int(len(df)*0.70)])
data_test = pd.DataFrame(df['Close'][int(len(df)*0.70): int(len(df))])

print(data_training.shape)
print(data_test.shape)

In [ ]:
data_training.head()

In [ ]:
data_test.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
data_training_array = scaler.fit_transform(data_training)
data_training_array

In [ ]:
x_train = []
y_train = []

for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i-100: i])
    y_train.append(data_training_array[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)  

In [ ]:
# ML Model
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(units = 50, activation = 'relu', return_sequences = True,
              input_shape = (x_train.shape[1], 1)))

model.add(Dropout(0.2))


model.add(LSTM(units = 60, activation = 'relu', return_sequences = True,))

model.add(Dropout(0.3))



model.add(LSTM(units = 80, activation = 'relu', return_sequences = True,))

model.add(Dropout(0.4))



model.add(LSTM(units = 120, activation = 'relu'))

model.add(Dropout(0.5))


model.add(Dense(units = 1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, epochs = 50)

In [ ]:
model.save('keras_model.h5')

In [ ]:
data_test.head()

In [ ]:
past_100_days = data_training.tail(100)


In [ ]:
final_df = past_100_days.append(data_test, ignore_index=True)

In [ ]:
final_df

In [ ]:
input_data = scaler.fit_transform(final_df)

In [ ]:
input_data

In [ ]:
input_data.shape

In [ ]:
x_test = []
y_test = []

for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i-100: i])
    y_test.append(input_data[i, 0])

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)
print(x_test.shape)

In [ ]:
print(y_test.shape)

In [ ]:
# Making Predictions

y_predicted = model.predict(x_test)

In [ ]:
y_predicted.shape

In [ ]:
y_test

In [ ]:
y_predicted

In [ ]:
scaler.scale_

In [ ]:
scale_factor = 1/0.00988704
y_predicted = y_predicted * scale_factor
y_test = y_test * scale_factor

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test, 'b', label = 'Original Price')
plt.plot(y_predicted, 'r', label = 'Predicted Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()
